This Jupyter notebook is dedicated to generating time series data for multiple configurations using the `TSBuilder` class from the `d2c.data_generation.builder` module. The notebook is structured to perform data generation in several phases, leveraging Python’s multiprocessing capabilities to handle multiple data generation tasks simultaneously.

### Breakdown of the Notebook:

1. **Setup and Imports:**
   - The notebook starts by importing necessary libraries such as `Pool` from `multiprocessing` and `TSBuilder` for building time series.
   - Necessary constants like `N_JOBS` (number of parallel jobs) are defined.

2. **Initial Data Generation:**
   - A `run_process` function is defined to handle the generation of time series data. This function attempts to build time series with specific parameters (like the number of variables, neighborhood size, noise standard deviation, etc.) and save them as pickle files.
   - A multiprocessing pool is set up to run these data generation tasks in parallel across various configurations (different processes, variable counts, neighborhood sizes, and noise levels).

3. **Verification of Generated Data:**
   - The notebook includes a check for any missing data files based on expected combinations of parameters. If any files are missing, their names are collected into a list.

4. **Re-Generation of Missing Data:**
   - The missing data files are then targeted for regeneration. Adjustments are made (like changing the seed and increasing the number of max attempts) to ensure successful generation.
   - Another round of multiprocessing is initiated specifically for the missing files.

5. **Final Verification:**
   - A final check is performed to ensure all expected data files are generated. If any files are still missing, they are targeted for another attempt with even more attempts and a different seed.

6. **Summary:**
   - At the end of the notebook, a check is performed to confirm the total number of files in the data directory, indicating the completion of the data generation process.

### Key Features:
- **Use of Multiprocessing:** To expedite data generation, the notebook utilizes the multiprocessing library to run multiple instances of data generation tasks simultaneously.
- **Error Handling:** Each data generation task includes try-except blocks to handle potential errors that might occur during the generation process.
- **Progressive Problem Solving:** The notebook incrementally addresses issues by adjusting parameters and retrying data generation for failed tasks.

This notebook serves as a comprehensive script for generating a large dataset of time series with varying characteristics, aiming to robustly handle errors and ensure complete generation across specified configurations.

In [1]:
# ! pip install ../.

In [36]:
# set wd 
import os
os.chdir('/home/jpalombarini/td2c/notebooks/paper_td2c/data')

In [37]:
from multiprocessing import Pool
from d2c.data_generation.builder import TSBuilder

In [44]:
N_JOBS = 55
def run_process(params):
    process, n_variables, max_neighborhood_size, noise_std = params
    try:
        tsbuilder = TSBuilder(observations_per_time_series=250, 
                              maxlags=5, 
                              n_variables=n_variables, 
                              time_series_per_process=40, 
                              processes_to_use=[process], 
                              noise_std=noise_std, 
                              max_neighborhood_size=max_neighborhood_size, 
                              seed=42, 
                              max_attempts=200,
                              verbose=True)

        tsbuilder.build()
        tsbuilder.to_pickle(f'/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std}.pkl')
        print(f'P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std} done')
    except ValueError as e:
        print(f'P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std} failed: {e}')

In [45]:
if __name__ == '__main__':
    parameters = [(process, n_variables, max_neighborhood_size, noise_std)
                  for process in [1, 2, 3] # , 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20
                  for n_variables in [5] # , 10, 25
                  for max_neighborhood_size in [2] # , 4, 8
                  for noise_std in [0.01]] # , 0.005, 0.001

    with Pool(processes=N_JOBS) as pool:
        pool.map(run_process, parameters)

P3_N5_Nj2_n0.01 done
P2_N5_Nj2_n0.01 done
P1_N5_Nj2_n0.01 done


Let's check any missing combinations

In [46]:
import os 
missing = []
for process in [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20]:
    for n_variables in [5,10,25]:
        for max_neighborhood_size in [2,4,8]:
            for noise_std in [0.01, 0.005, 0.001]:
                filename = f'/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std}.pkl'
                if not os.path.exists(filename):
                    missing.append(filename)


In [47]:
missing

['/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj2_n0.005.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj2_n0.001.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj4_n0.01.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj4_n0.005.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj4_n0.001.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj8_n0.01.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj8_n0.005.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N5_Nj8_n0.001.pkl',
 '/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data/P1_N10_Nj2_n0.01.pkl',
 '/home/jpalombarini/td2c/notebooks/contribution

Now we focus on recreating the missing ones, by changing the seeds and increasing the number of max_attempts

In [10]:
from multiprocessing import Pool
from d2c.data_generation.builder import TSBuilder
N_JOBS = 55
def run_process(params):
    process, n_variables, max_neighborhood_size, noise_std = params
    try:# we change the seed and increase the max_attempts
        tsbuilder = TSBuilder(observations_per_time_series=250, 
                              maxlags=5, 
                              n_variables=n_variables, 
                              time_series_per_process=40, 
                              processes_to_use=[process], 
                              noise_std=noise_std, 
                              max_neighborhood_size=max_neighborhood_size, 
                              seed=24, 
                              max_attempts=400,
                              verbose=True)

        tsbuilder.build()
        tsbuilder.to_pickle(f'./data/P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std}.pkl')
        print(f'P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std} done')
    except ValueError as e:
        print(f'P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std} failed: {e}')

if __name__ == '__main__':
    parameters = []
    for missing_file in missing:
        process = int(missing_file.split('/')[-1].split('_')[0][1:])
        n_variables = int(missing_file.split('/')[-1].split('_')[1][1:])
        max_neighborhood_size = int(missing_file.split('/')[-1].split('_')[2][2:])
        noise_std = float(missing_file.split('/')[-1].split('_')[3][1:-4])
        parameters.append((process, n_variables, max_neighborhood_size, noise_std))


    with Pool(processes=N_JOBS) as pool:
        pool.map(run_process, parameters)

P6_N25_Nj2_n0.001 failed: Failed to generate valid TS for model 6, TS index 1 after 400 attempts. Try again with a different seed.

P16_N25_Nj2_n0.001 failed: Failed to generate valid TS for model 16, TS index 10 after 400 attempts. Try again with a different seed.
P6_N25_Nj8_n0.001 failed: Failed to generate valid TS for model 6, TS index 8 after 400 attempts. Try again with a different seed.
P14_N25_Nj8_n0.001 failed: Failed to generate valid TS for model 14, TS index 17 after 400 attempts. Try again with a different seed.
P16_N25_Nj8_n0.001 failed: Failed to generate valid TS for model 16, TS index 24 after 400 attempts. Try again with a different seed.
P20_N25_Nj2_n0.001 done
P14_N25_Nj2_n0.001 done
P13_N25_Nj8_n0.001 failed: Failed to generate valid TS for model 13, TS index 18 after 400 attempts. Try again with a different seed.
P16_N25_Nj4_n0.001 done
P20_N25_Nj4_n0.001 done
P11_N25_Nj2_n0.001 done
P20_N25_Nj8_n0.001 failed: Failed to generate valid TS for model 20, TS index 33 

Let's check what is still missing

In [11]:
import os 
missing = []
for process in [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20]:
    for n_variables in [5,10,25]:
        for max_neighborhood_size in [2,4,8]:
            for noise_std in [0.01, 0.005, 0.001]:
                filename = f'./data/P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std}.pkl'
                if not os.path.exists(filename):
                    missing.append(filename)


In [12]:
missing

['./data/P6_N25_Nj2_n0.001.pkl',
 './data/P6_N25_Nj4_n0.001.pkl',
 './data/P6_N25_Nj8_n0.001.pkl',
 './data/P13_N25_Nj4_n0.001.pkl',
 './data/P13_N25_Nj8_n0.001.pkl',
 './data/P14_N25_Nj8_n0.001.pkl',
 './data/P16_N25_Nj2_n0.001.pkl',
 './data/P16_N25_Nj8_n0.001.pkl',
 './data/P20_N25_Nj8_n0.001.pkl']

We try one last time with even more max_attemps and a different seed.

In [13]:
from multiprocessing import Pool
from d2c.data_generation.builder import TSBuilder
N_JOBS = len(missing)
def run_process(params):
    process, n_variables, max_neighborhood_size, noise_std = params
    try:# we change the seed and increase the max_attempts
        tsbuilder = TSBuilder(observations_per_time_series=250, 
                              maxlags=5, 
                              n_variables=n_variables, 
                              time_series_per_process=40, 
                              processes_to_use=[process], 
                              noise_std=noise_std, 
                              max_neighborhood_size=max_neighborhood_size, 
                              seed=0, 
                              max_attempts=1000,
                              verbose=True)

        tsbuilder.build()
        tsbuilder.to_pickle(f'./data/P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std}.pkl')
        print(f'P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std} done')
    except ValueError as e:
        print(f'P{process}_N{n_variables}_Nj{max_neighborhood_size}_n{noise_std} failed: {e}')

if __name__ == '__main__':
    parameters = []
    for missing_file in missing:
        process = int(missing_file.split('/')[-1].split('_')[0][1:])
        n_variables = int(missing_file.split('/')[-1].split('_')[1][1:])
        max_neighborhood_size = int(missing_file.split('/')[-1].split('_')[2][2:])
        noise_std = float(missing_file.split('/')[-1].split('_')[3][1:-4])
        parameters.append((process, n_variables, max_neighborhood_size, noise_std))
# what this if statement does is to run the function run_process in parallel for each parameter in the list parameters
# if __name__ == '__main__': is a python thing that allows you to run the code in the if statement only if you run the script directly

    with Pool(processes=N_JOBS) as pool:
        pool.map(run_process, parameters)

P16_N25_Nj2_n0.001 done

P20_N25_Nj8_n0.001 done
P14_N25_Nj8_n0.001 done
P16_N25_Nj8_n0.001 done
P6_N25_Nj2_n0.001 done
P13_N25_Nj4_n0.001 done
P6_N25_Nj4_n0.001 done
P13_N25_Nj8_n0.001 done
P6_N25_Nj8_n0.001 done


All time series have been generated correctly

In [48]:
len(os.listdir('/home/jpalombarini/td2c/notebooks/contributions/Real_data_validation/useful_data'))

3